# Alberta Education Spending Comparison: 2012–2025

This notebook compares Alberta's education spending across budget years,
using data extracted from:
- **Budget 2012 & 2013** fiscal plan PDFs (via pdfplumber)
- **Budget 2023, 2024 & 2025** expense tables (Excel files)

All figures are **operating expense in millions of CAD**.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import numpy as np
import seaborn as sns

# ============================================================
# GLOBAL STYLE — Dark Infographic Theme
# ============================================================
sns.set_theme(style='darkgrid', context='notebook', font_scale=1.15)
plt.rcParams.update({
    'figure.facecolor': '#0D1117',
    'axes.facecolor':   '#161B22',
    'axes.edgecolor':   '#30363D',
    'axes.labelcolor':  '#E6EDF3',
    'text.color':       '#E6EDF3',
    'xtick.color':      '#8B949E',
    'ytick.color':      '#8B949E',
    'grid.color':       '#21262D',
    'grid.alpha':       0.6,
    'font.family':      'sans-serif',
    'savefig.facecolor':'#0D1117',
})

# Vibrant palette
PALETTE = {
    'Budget 2012': '#00D4AA',   # teal
    'Budget 2013': '#00B4D8',   # cyan
    'Budget 2023': '#FFB703',   # amber
    'Budget 2024': '#FB8500',   # orange
    'Budget 2025': '#E63946',   # red
}
K12_COLOR = '#58A6FF'           # bright blue
PS_COLOR  = '#56D364'           # bright green
ACCENT    = '#F778BA'           # pink accent

print('✓ Seaborn dark infographic theme loaded')

In [ ]:
# ============================================================
# 1. BUILD THE EDUCATION SPENDING DATAFRAME
# ============================================================
data = [
    # --- Budget 2012 (Fiscal Plan 2012–15 PDF, pp. 27-28, 114) ---
    {'Budget_Year': 'Budget 2012', 'Fiscal_Year': '2010-11', 'Type': 'Actual',
     'K12_Operating_Expense': 5609, 'PostSecondary_Operating_Expense': 2683, 'Source': 'PDF'},
    {'Budget_Year': 'Budget 2012', 'Fiscal_Year': '2011-12', 'Type': 'Forecast',
     'K12_Operating_Expense': 5977, 'PostSecondary_Operating_Expense': 2780, 'Source': 'PDF'},
    {'Budget_Year': 'Budget 2012', 'Fiscal_Year': '2012-13', 'Type': 'Estimate',
     'K12_Operating_Expense': 6179, 'PostSecondary_Operating_Expense': 2856, 'Source': 'PDF'},
    {'Budget_Year': 'Budget 2012', 'Fiscal_Year': '2013-14', 'Type': 'Target',
     'K12_Operating_Expense': 6387, 'PostSecondary_Operating_Expense': 2983, 'Source': 'PDF'},
    {'Budget_Year': 'Budget 2012', 'Fiscal_Year': '2014-15', 'Type': 'Target',
     'K12_Operating_Expense': 6625, 'PostSecondary_Operating_Expense': 3039, 'Source': 'PDF'},

    # --- Budget 2013 (Fiscal Plan 2013–16 PDF, pp. 132-133) ---
    {'Budget_Year': 'Budget 2013', 'Fiscal_Year': '2011-12', 'Type': 'Actual',
     'K12_Operating_Expense': 5946, 'PostSecondary_Operating_Expense': 2700, 'Source': 'PDF'},
    {'Budget_Year': 'Budget 2013', 'Fiscal_Year': '2012-13', 'Type': 'Forecast',
     'K12_Operating_Expense': 6166, 'PostSecondary_Operating_Expense': 2782, 'Source': 'PDF'},
    {'Budget_Year': 'Budget 2013', 'Fiscal_Year': '2013-14', 'Type': 'Estimate',
     'K12_Operating_Expense': 6210, 'PostSecondary_Operating_Expense': 2682, 'Source': 'PDF'},
    {'Budget_Year': 'Budget 2013', 'Fiscal_Year': '2014-15', 'Type': 'Target',
     'K12_Operating_Expense': 6353, 'PostSecondary_Operating_Expense': 2728, 'Source': 'PDF'},
    {'Budget_Year': 'Budget 2013', 'Fiscal_Year': '2015-16', 'Type': 'Target',
     'K12_Operating_Expense': 6546, 'PostSecondary_Operating_Expense': 2754, 'Source': 'PDF'},

    # --- Budget 2023 (expense_tables_2023-26.xlsx) ---
    {'Budget_Year': 'Budget 2023', 'Fiscal_Year': '2022-23', 'Type': 'Budget',
     'K12_Operating_Expense': 8403, 'PostSecondary_Operating_Expense': 5346, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2023', 'Fiscal_Year': '2022-23 (F)', 'Type': 'Forecast',
     'K12_Operating_Expense': 8477, 'PostSecondary_Operating_Expense': 5422, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2023', 'Fiscal_Year': '2023-24', 'Type': 'Estimate',
     'K12_Operating_Expense': 8836, 'PostSecondary_Operating_Expense': 5604, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2023', 'Fiscal_Year': '2024-25', 'Type': 'Target',
     'K12_Operating_Expense': 9036, 'PostSecondary_Operating_Expense': 5730, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2023', 'Fiscal_Year': '2025-26', 'Type': 'Target',
     'K12_Operating_Expense': 9162, 'PostSecondary_Operating_Expense': 5862, 'Source': 'Excel'},

    # --- Budget 2024 (expense_tables_2024-27.xlsx) ---
    {'Budget_Year': 'Budget 2024', 'Fiscal_Year': '2023-24', 'Type': 'Budget',
     'K12_Operating_Expense': 8836, 'PostSecondary_Operating_Expense': 5779, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2024', 'Fiscal_Year': '2023-24 (F)', 'Type': 'Forecast',
     'K12_Operating_Expense': 8859, 'PostSecondary_Operating_Expense': 6116, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2024', 'Fiscal_Year': '2024-25', 'Type': 'Estimate',
     'K12_Operating_Expense': 9252, 'PostSecondary_Operating_Expense': 6305, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2024', 'Fiscal_Year': '2025-26', 'Type': 'Target',
     'K12_Operating_Expense': 9471, 'PostSecondary_Operating_Expense': 6471, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2024', 'Fiscal_Year': '2026-27', 'Type': 'Target',
     'K12_Operating_Expense': 9561, 'PostSecondary_Operating_Expense': 6598, 'Source': 'Excel'},

    # --- Budget 2025 (expense_tables_2025-28.xlsx) ---
    {'Budget_Year': 'Budget 2025', 'Fiscal_Year': '2023-24', 'Type': 'Actual',
     'K12_Operating_Expense': 8878, 'PostSecondary_Operating_Expense': 6233, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2025', 'Fiscal_Year': '2024-25', 'Type': 'Budget',
     'K12_Operating_Expense': 9252, 'PostSecondary_Operating_Expense': 6305, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2025', 'Fiscal_Year': '2024-25 (F)', 'Type': 'Forecast',
     'K12_Operating_Expense': 9457, 'PostSecondary_Operating_Expense': 6628, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2025', 'Fiscal_Year': '2025-26', 'Type': 'Estimate',
     'K12_Operating_Expense': 9883, 'PostSecondary_Operating_Expense': 6635, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2025', 'Fiscal_Year': '2026-27', 'Type': 'Target',
     'K12_Operating_Expense': 10285, 'PostSecondary_Operating_Expense': 6629, 'Source': 'Excel'},
    {'Budget_Year': 'Budget 2025', 'Fiscal_Year': '2027-28', 'Type': 'Target',
     'K12_Operating_Expense': 10708, 'PostSecondary_Operating_Expense': 6629, 'Source': 'Excel'},
]

df = pd.DataFrame(data)
df['Total_Education_Expense'] = df['K12_Operating_Expense'] + df['PostSecondary_Operating_Expense']

# Headline: one estimate row per budget
headline = df[df['Type'] == 'Estimate'].copy()
headline = headline[['Budget_Year', 'Fiscal_Year', 'K12_Operating_Expense',
                      'PostSecondary_Operating_Expense', 'Total_Education_Expense']]
headline = headline.rename(columns={
    'K12_Operating_Expense': 'K-12 ($M)',
    'PostSecondary_Operating_Expense': 'Post-Secondary ($M)',
    'Total_Education_Expense': 'Total ($M)'
}).reset_index(drop=True)

# Growth table
b2012 = headline[headline['Budget_Year'] == 'Budget 2012'].iloc[0]
b2025 = headline[headline['Budget_Year'] == 'Budget 2025'].iloc[0]
growth = pd.DataFrame({
    'Category': ['K-12', 'Post-Secondary', 'Total Education'],
    'Budget 2012 ($M)': [b2012['K-12 ($M)'], b2012['Post-Secondary ($M)'], b2012['Total ($M)']],
    'Budget 2025 ($M)': [b2025['K-12 ($M)'], b2025['Post-Secondary ($M)'], b2025['Total ($M)']],
})
growth['Change ($M)'] = growth['Budget 2025 ($M)'] - growth['Budget 2012 ($M)']
growth['Change (%)'] = ((growth['Budget 2025 ($M)'] / growth['Budget 2012 ($M)']) - 1) * 100

print('=== Data loaded: headline rows per budget ===')
display(headline.style.format({
    'K-12 ($M)': '${:,.0f}',
    'Post-Secondary ($M)': '${:,.0f}',
    'Total ($M)': '${:,.0f}'
}))

In [ ]:
# ============================================================
# INFOGRAPHIC 1: Side-by-Side K-12 vs Post-Secondary Bar Chart
# ============================================================

# Melt headline for seaborn grouped barplot
plot_df = headline.melt(
    id_vars=['Budget_Year', 'Fiscal_Year'],
    value_vars=['K-12 ($M)', 'Post-Secondary ($M)'],
    var_name='Category', value_name='Spending ($M)'
)

fig, ax = plt.subplots(figsize=(14, 8))

sns.barplot(
    data=plot_df, x='Budget_Year', y='Spending ($M)', hue='Category',
    palette=[K12_COLOR, PS_COLOR], edgecolor='#0D1117', linewidth=1.5,
    saturation=1.0, ax=ax
)

# Value labels on each bar
for container in ax.containers:
    for bar in container:
        h = bar.get_height()
        if h > 0:
            ax.text(bar.get_x() + bar.get_width()/2, h + 100,
                    f'${h:,.0f}M', ha='center', va='bottom',
                    fontsize=9, fontweight='bold', color='#E6EDF3')

ax.set_title('Alberta Education Operating Expense\nK-12 vs Post-Secondary (Headline Estimate)',
             fontsize=18, fontweight='bold', pad=20, color='white')
ax.set_xlabel('', fontsize=12)
ax.set_ylabel('Operating Expense ($ millions)', fontsize=13, labelpad=10)
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x:,.0f}'))
ax.set_ylim(0, plot_df['Spending ($M)'].max() * 1.18)

# Bright legend
leg = ax.legend(title='Category', fontsize=12, title_fontsize=13,
                loc='upper left', framealpha=0.7, edgecolor='#30363D')
leg.get_frame().set_facecolor('#161B22')
for text in leg.get_texts():
    text.set_color('#E6EDF3')
leg.get_title().set_color('#E6EDF3')

ax.tick_params(axis='x', rotation=15)
sns.despine(left=True, bottom=True)

plt.tight_layout()
plt.savefig('budget_data/infographic_k12_vs_postsec.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved → budget_data/infographic_k12_vs_postsec.png')

In [ ]:
# ============================================================
# INFOGRAPHIC 2: Stacked Horizontal Bar — Total Education
# ============================================================

fig, ax = plt.subplots(figsize=(14, 6))

budgets = headline['Budget_Year'].values
k12     = headline['K-12 ($M)'].values
ps      = headline['Post-Secondary ($M)'].values
y_pos   = np.arange(len(budgets))

# Draw horizontal stacked bars
bars_k12 = ax.barh(y_pos, k12, height=0.55, label='K-12',
                   color=K12_COLOR, edgecolor='#0D1117', linewidth=1.2)
bars_ps  = ax.barh(y_pos, ps, height=0.55, left=k12, label='Post-Secondary',
                   color=PS_COLOR, edgecolor='#0D1117', linewidth=1.2)

# Labels inside bars
for i in range(len(budgets)):
    # K-12 label
    ax.text(k12[i]/2, y_pos[i], f'${k12[i]:,.0f}M',
            ha='center', va='center', fontsize=10, fontweight='bold', color='#0D1117')
    # Post-secondary label
    ax.text(k12[i] + ps[i]/2, y_pos[i], f'${ps[i]:,.0f}M',
            ha='center', va='center', fontsize=10, fontweight='bold', color='#0D1117')
    # Total at end
    total = k12[i] + ps[i]
    ax.text(total + 150, y_pos[i], f'Total: ${total:,.0f}M',
            ha='left', va='center', fontsize=11, fontweight='bold', color=ACCENT)

ax.set_yticks(y_pos)
ax.set_yticklabels(budgets, fontsize=13, fontweight='bold')
ax.invert_yaxis()
ax.set_xlabel('Operating Expense ($ millions)', fontsize=13, labelpad=10)
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x:,.0f}'))
ax.set_title('Total Education Spending by Budget Year\n(K-12 + Post-Secondary)',
             fontsize=18, fontweight='bold', pad=20, color='white')

leg = ax.legend(fontsize=12, loc='lower right', framealpha=0.7, edgecolor='#30363D')
leg.get_frame().set_facecolor('#161B22')
for text in leg.get_texts():
    text.set_color('#E6EDF3')

sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig('budget_data/infographic_total_stacked.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved → budget_data/infographic_total_stacked.png')

In [ ]:
# ============================================================
# INFOGRAPHIC 3: Growth Waterfall — 2012 vs 2025
# ============================================================

fig, ax = plt.subplots(figsize=(12, 7))

categories = growth['Category'].values
vals_2012  = growth['Budget 2012 ($M)'].values
vals_2025  = growth['Budget 2025 ($M)'].values
change_pct = growth['Change (%)'].values

x = np.arange(len(categories))
width = 0.32

bars1 = ax.bar(x - width/2, vals_2012, width, label='Budget 2012 (2012-13)',
               color='#00D4AA', edgecolor='#0D1117', linewidth=1.5)
bars2 = ax.bar(x + width/2, vals_2025, width, label='Budget 2025 (2025-26)',
               color='#E63946', edgecolor='#0D1117', linewidth=1.5)

# Value + growth annotations
for i in range(len(categories)):
    # 2012 value
    ax.text(x[i] - width/2, vals_2012[i] + 150,
            f'${vals_2012[i]:,.0f}M', ha='center', va='bottom',
            fontsize=10, fontweight='bold', color='#00D4AA')
    # 2025 value
    ax.text(x[i] + width/2, vals_2025[i] + 150,
            f'${vals_2025[i]:,.0f}M', ha='center', va='bottom',
            fontsize=10, fontweight='bold', color='#E63946')
    # Growth % arrow annotation
    mid_y = max(vals_2012[i], vals_2025[i]) + 800
    ax.annotate(f'+{change_pct[i]:.0f}%',
                xy=(x[i], mid_y), fontsize=16, fontweight='bold',
                ha='center', va='bottom', color='#FFB703',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='#21262D',
                          edgecolor='#FFB703', alpha=0.9))

ax.set_xticks(x)
ax.set_xticklabels(categories, fontsize=14, fontweight='bold')
ax.set_ylabel('Operating Expense ($ millions)', fontsize=13, labelpad=10)
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x:,.0f}'))
ax.set_ylim(0, max(vals_2025) * 1.35)
ax.set_title('Education Spending Growth: 2012 → 2025\n13 Years of Change',
             fontsize=18, fontweight='bold', pad=20, color='white')

leg = ax.legend(fontsize=13, loc='upper left', framealpha=0.7, edgecolor='#30363D')
leg.get_frame().set_facecolor('#161B22')
for text in leg.get_texts():
    text.set_color('#E6EDF3')

sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig('budget_data/infographic_growth_2012_vs_2025.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved → budget_data/infographic_growth_2012_vs_2025.png')

In [ ]:
# ============================================================
# INFOGRAPHIC 4: Donut Charts — Budget Composition
# ============================================================

fig, axes = plt.subplots(1, 5, figsize=(22, 5))
fig.suptitle('K-12 vs Post-Secondary Share by Budget Year',
             fontsize=18, fontweight='bold', color='white', y=1.05)

donut_colors = [K12_COLOR, PS_COLOR]

for ax, (_, row) in zip(axes, headline.iterrows()):
    vals = [row['K-12 ($M)'], row['Post-Secondary ($M)']]
    total = sum(vals)
    pcts = [v/total*100 for v in vals]

    wedges, texts = ax.pie(
        vals, colors=donut_colors, startangle=90,
        wedgeprops=dict(width=0.4, edgecolor='#0D1117', linewidth=2)
    )

    # Center text
    ax.text(0, 0.08, f'${total:,.0f}M', ha='center', va='center',
            fontsize=12, fontweight='bold', color='white')
    ax.text(0, -0.12, 'Total', ha='center', va='center',
            fontsize=9, color='#8B949E')

    # Title below
    budget_label = row['Budget_Year'].replace('Budget ', "'")
    ax.set_title(row['Budget_Year'], fontsize=12, fontweight='bold',
                 color=PALETTE.get(row['Budget_Year'], 'white'), pad=12)

    # Percentage annotations
    ax.text(0, -0.65, f'K-12: {pcts[0]:.0f}%  |  Post-Sec: {pcts[1]:.0f}%',
            ha='center', va='center', fontsize=8, color='#8B949E')

plt.tight_layout()
plt.savefig('budget_data/infographic_donut_composition.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved → budget_data/infographic_donut_composition.png')

In [ ]:
# ============================================================
# INFOGRAPHIC 5: Heatmap — All Budget Years × Categories
# ============================================================

heat_data = headline.set_index('Budget_Year')[['K-12 ($M)', 'Post-Secondary ($M)', 'Total ($M)']]

fig, ax = plt.subplots(figsize=(10, 5))

sns.heatmap(
    heat_data, annot=True, fmt=',.0f', cmap='YlOrRd',
    linewidths=2, linecolor='#0D1117', cbar_kws={'label': '$ millions'},
    annot_kws={'fontsize': 13, 'fontweight': 'bold'}, ax=ax
)

ax.set_title('Education Spending Heatmap\n(Higher = More Spending)',
             fontsize=16, fontweight='bold', pad=15, color='white')
ax.set_ylabel('')
ax.tick_params(axis='y', rotation=0)
ax.tick_params(axis='x', rotation=15)

# Fix colorbar label color
cbar = ax.collections[0].colorbar
cbar.ax.yaxis.label.set_color('#E6EDF3')
cbar.ax.tick_params(colors='#8B949E')

plt.tight_layout()
plt.savefig('budget_data/infographic_heatmap.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved → budget_data/infographic_heatmap.png')

In [ ]:
# ============================================================
# INFOGRAPHIC 6: Lollipop Chart — Growth % by Category
# ============================================================

fig, ax = plt.subplots(figsize=(10, 5))

colors_lollipop = ['#58A6FF', '#56D364', '#F778BA']
categories = growth['Category'].values
pcts = growth['Change (%)'].values
y_pos = np.arange(len(categories))

# Horizontal lollipop
for i in range(len(categories)):
    ax.hlines(y=y_pos[i], xmin=0, xmax=pcts[i],
              color=colors_lollipop[i], linewidth=3, alpha=0.8)
    ax.scatter(pcts[i], y_pos[i], color=colors_lollipop[i],
              s=250, zorder=5, edgecolor='#0D1117', linewidth=2)
    ax.text(pcts[i] + 3, y_pos[i],
            f'+{pcts[i]:.0f}%  (+${growth["Change ($M)"].iloc[i]:,.0f}M)',
            va='center', fontsize=13, fontweight='bold', color=colors_lollipop[i])

ax.set_yticks(y_pos)
ax.set_yticklabels(categories, fontsize=14, fontweight='bold')
ax.set_xlabel('Growth (%)', fontsize=13, labelpad=10)
ax.set_xlim(-5, max(pcts) * 1.4)
ax.set_title('Education Spending Growth 2012 → 2025\nPercentage Increase by Category',
             fontsize=16, fontweight='bold', pad=15, color='white')
ax.axvline(x=0, color='#30363D', linewidth=1)

sns.despine(left=True, bottom=True)
plt.tight_layout()
plt.savefig('budget_data/infographic_lollipop_growth.png', dpi=150, bbox_inches='tight')
plt.show()
print('Saved → budget_data/infographic_lollipop_growth.png')

In [ ]:
# ============================================================
# EXPORT DATA TO CSV
# ============================================================
df.to_csv('budget_data/education_spending_comparison.csv', index=False)
headline.to_csv('budget_data/education_spending_headline.csv', index=False)
growth.to_csv('budget_data/education_spending_growth.csv', index=False)

print('\n✓ All infographics saved to budget_data/')
print('✓ CSV exports:')
print('   • education_spending_comparison.csv  (full data)')
print('   • education_spending_headline.csv    (estimate rows only)')
print('   • education_spending_growth.csv      (2012 vs 2025 growth)')